### Demo notebook for transmission analysis

* Calculate which branches or transmission lines are congested by counting the fraction of total hours that the power flow is 0.75, 0.8 or 0.9 of the branch capacity. Z-score and p-values are calculated for the 0.75 case, since 0.75 is also the value used by WECC (useful for validation).


In [1]:
import pandas as pd
import numpy as np

from postreise.analyze import transmission
from postreise.process import transferdata as td
from westernintnet.westernintnet import win_data

c:\program files\python\python36\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Done loading


* Power flow outputs are usually saved as pkl files. These are large files and are not saved in github.

In [2]:
od = td.OutputData()
pf = od.get_data('western_scenario_Update01', 'PF')

Use  C:\Users\brdo\scenario_data\  to save/load local scenario data.


In [3]:
win_data.pf_base = pf
win_data.branches.index = pf.columns

# Create Capacity column; Note that rateA == 0 represents low impedance lines
# Denoting rateA==0 is simply a signal to MatPower that there are no current constraints on the line.
win_data.branches.loc[win_data.branches.rateA != 0, 'Capacity'] = win_data.branches['rateA']
win_data.branches.loc[win_data.branches.rateA == 0, 'Capacity'] = 99999.

# Normalize power flow values to line capacity
win_data.congestion_base = win_data.pf_base.div(win_data.branches['Capacity']).apply(np.abs)  

# Create congestion statistics file
win_data.congestion_stats_base = transmission.generate_cong_stats(win_data.congestion_base,
                                                                  win_data.branches,
                                                                  '../data/congestion_stats_base_demo')

In [4]:
win_data.congestion_stats_base.loc[win_data.congestion_stats_base['pvalue'] < 0.05].sort_values(
    ['pvalue', 'zscore'], ascending=False).head(10)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
line,,,,,,,,,,,
12226,392,0,0,6,34,0,6,40,0.000000,2.615872,4.449990e-03
11438,169,0,7,18,17,7,25,42,0.000000,3.004657,1.329401e-03
11668,279,0,0,2,40,0,2,42,53.208235,3.004657,1.329401e-03
10335,476,0,0,1,45,0,1,46,48.364319,3.782227,7.771575e-05
9877,214,0,0,27,22,0,27,49,44.630752,4.365404,6.344391e-06
11106,567,0,0,6,43,0,6,49,111.365148,4.365404,6.344391e-06
3561,222,0,3,18,29,3,21,50,1.709758,4.559797,2.560157e-06
5603,268,0,0,4,47,0,4,51,43.555793,4.754189,9.962218e-07
11107,497,0,0,6,52,0,6,58,19.233258,6.114936,4.829778e-10
